In [47]:
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import re
import pandas as pd
import json
import time

driver_loc='/opt/homebrew/bin/chromedriver'

## Next step is to read tourney_links.json and process each tournament

In [37]:
with open('pga_data/data_files/tourney_links.json', 'r') as f:
    links = json.load(f)

In [53]:
def get_tourney_details(link, name):
    """Get details of a specific tournament.
    
    Parameters
    ----------
    link : str
        This string should be a url that leads to a tournemnt from pgatour.com.
    name : str
        The name of the tournament
    """

    start_url = link + '/course-stats'

    # Initiate webdriver
    s = Service(driver_loc)
    driver = webdriver.Chrome(service=s)
    driver.get(start_url)


    # Initialize dictionary
    course = {}

    # Select all holes
    button_class = 'css-qchwq2'
    try:
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, button_class)))
    except NoSuchElementException:
        raise NoSuchElementException(f'Unable to select Course Stats for {name}')
    
    hole_locs_bttn = driver.find_element(By.CLASS_NAME, button_class)
    hole_locs_bttn.click()

    # All holes are now selected.
    # TODO: Determine if it is necessary to extract pin location information from these images

    # Get course name
    soup = BeautifulSoup(driver.page_source, 'lxml')
    course_class = 'chakra-text css-mo47gj'
    try:
        course_name = soup.find(attrs={'class': course_class}).text
    except:
        print(f'Course name unable to be found')
        course_name = None
    course['course_name'] = course_name
    


    # Get hole stats

    # Get course details


    # Closeout driver
    # driver.quit()
    # time.sleep(10)
    return course


tourney = links[0]  # Fortinet Championship
soup = get_tourney_details(tourney['tournament_link'], tourney['tournament_name'])

In [54]:
soup

{'course_name': 'Silverado Resort and Spa (North Course)'}